# STEP TWO! WORK IN PROGRESS! CANT GET IT TO RUN!

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   WARNING: Do not use the development server in a production environment.
   Use a production WSGI server instead.
 * Debug mode: on

 * Restarting with stat

An exception has occurred, use %tb to see the full traceback.

SystemExit: 1




## Step 2 - Climate App

Now that you have completed your initial analysis, 
design a Flask API based on the queries that you have just developed.
Use FLASK to create your routes.

Routes
/
Home page.
List all routes that are available.
/api/v1.0/precipitation

Convert the query results to a Dictionary using date as the key and prcp as the value.

Return the JSON representation of your dictionary.

/api/v1.0/stations

Return a JSON list of stations from the dataset.

/api/v1.0/tobs

query for the dates and temperature observations from a year from the last data point.

Return a JSON list of Temperature Observations (tobs) for the previous year.

/api/v1.0/<start> and /api/v1.0/<start>/<end>
    
Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or 

start-end range.

When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.


Hints
You will need to join the station and measurement tables for some of the analysis queries.
Use Flask jsonify to convert your API data into a valid JSON response object.

In [6]:
# Import Dependencies
import datetime as datetime
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

#Create Engine
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()

# Reflect tables
Base.prepare(engine, reflect=True)

#Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

app = Flask(__name__)



@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/&ltstart&gt, for example, /api/v1.0/yyyy-mm-dd<br/>"
        f"/api/v1.0/&ltstart&gt/&ltend&gt, for example, /api/v1.0/yyyy-mm-dd/yyyy-mm-dd"
)


if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

In [ ]:
# Define Routes
base_date = datetime.datetime.strptime("2017-08-23", "%Y-%m-%d")
numdays = 365
date_list = [base_date - datetime.timedelta(days=x) for x in range(0, numdays)]

# Converting them to a list of strings
str_dates = []
for date in date_list:
    new_date = date.strftime("%Y-%m-%d")
    str_dates.append(new_date)

##Convert the query results to a Dictionary using date as the key and prcp as the value.
##Return the JSON representation of your dictionary.    
    
@app.route("/api.v1.0/precipitation")

def precipitation():

date_start = dt.datetime.strftime(py_start, '%Y-%m-%d')
date_end = dt.datetime.strftime(py_end, '%Y-%m-%d')

sel = [Station.station, 
       Station.name, 
       Station.latitude, 
       Station.longitude, 
       Station.elevation, 
       func.sum(Measurement.prcp)]
result = session.query(*sel).\
    filter(Measurement.station == Station.station).\
    filter(Measurement.date >= date_start).\
    filter(Measurement.date <= date_end).\
    group_by(Station.name).order_by(func.sum(Measurement.prcp).desc()).all()
print (result)    
    
    
    ed_date = session.query(func.max(Measurement.date)).all()
    end_date = ed_date[0][0]

        # Perform a query to retrieve the data and precipitation scores
    start = dt.datetime.strftime(dt.datetime.strptime(end_date, '%Y-%m-%d') - dt.timedelta(days=365), '%Y-%m-%d')
    results = session.query(Measurement.date,Measurement.prcp).\
            filter(Measurement.date > start).order_by(Measurement.date).all()
        # Save the query results as a Pandas DataFrame and set the index to the date column
    prcp_df=pd.DataFrame(results)
    prcp_df.set_index('date').head()

        # Sort the dataframe by date
    prcp_df = prcp_df.sort_values(by=['date'])
    prcp_data =[]
    for date in results:
        prcp_dict ={}
        prcp_dict["date"]=date.date
        prcp_dict["prcp"]=date.prcp
        prcp_data.append(prcp_dict)

    return jsonify(prcp_data)




##Return a JSON list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    sts= session.query(func.count(distinct(Measurement.station))).all()
    stations = sts[0][0]
    print(f"There are {stations} stations.")
    
    results = session.query(Measurement.station,Station.name,func.count(Measurement.tobs).label('count')).\
        filter(Measurement.station==Station.station).\
        group_by(Measurement.station,Station.name).order_by('count DESC').all()

    station_data = []
    for station in results:
        station_dict = {}
        station_dict["Station"] = station.station
        station_dict["Name"] = station.name       
        station_dict["Count"] = station.count
        
        station_data.append(station_dict)

    return jsonify(station_data)

##query for the dates and temperature observations from a year from the last data point.
@app.route("/api/v1.0/tobs")
def tobs():
    # Query 
    ed_date = session.query(func.max(Measurement.date)).all()
    end_date = ed_date[0][0]

        # Perform a query to retrieve the data and precipitation scores
    start = dt.datetime.strftime(dt.datetime.strptime(end_date, '%Y-%m-%d') - dt.timedelta(days=365), '%Y-%m-%d')
    
    results = session.query(Measurement).filter(Measurement.date > start)

    temp_data = []
    for day in results:
        temp_dict = {}
        temp_dict[day.date] = day.tobs
        temp_data.append(temp_dict)

    return jsonify(temp_data)
##Routes / Home page. List all routes that are available. /api/v1.0/precipitation
@app.route("/api/v1.0/<start>")
def temperature_s(start_date, end_date):

 
    """TMIN, TAVG, and TMAX for a list of dates.   
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d        
    Returns:
        TMIN, TAVE, and TMAX
    """
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).\
        filter(Measurement.date >= start_date).\
        filter(Measurement.date <= end_date).all()

    # Grabbing avg, min & max temps    
    temp_avg = session.query(func.avg(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]
    temp_min = session.query(func.min(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]
    temp_max = session.query(func.max(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]

    # Dictionary of temperatures
    temp_dict = {}
    temp_dict["Average Temperature"] = temp_avg
    temp_dict["Minimum Temperature"] = temp_min
    temp_dict["Maximum Temperature"] = temp_max

    return jsonify(temp_dict)

@app.route("/api/v1.0/<start>/<end>")
def temperature(start, end):

    # Set start and end dates for date range
    startDate = datetime.datetime.strptime(start, "%Y-%m-%d")
    endDate = datetime.datetime.strptime(end, "%Y-%m-%d")

    # Date range
    # Getting date range
    delta = endDate - startDate
    date_range = []
    for i in range(delta.days + 1):
        date_range.append(startDate + timedelta(days=i))
    
    # Converting to strings to filter
    str_date_range = []
    for date in date_range:
        new_date = date.strftime("%Y-%m-%d")
        str_date_range.append(new_date)

    # Grabbing avg, min & max temps    
    temp_avg = session.query(func.avg(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]
    temp_min = session.query(func.min(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]
    temp_max = session.query(func.max(Measurement.tobs))\
                .filter(Measurement.date.in_(str_date_range))[0][0]

    # Dictionary of temperatures
    temp_dict = {}
    temp_dict["Average Temperature"] = temp_avg
    temp_dict["Minimum Temperature"] = temp_min
    temp_dict["Maximum Temperature"] = temp_max

    return jsonify(temp_dict)
